In [94]:
!pip install requests
!pip install groq

In [95]:
from IPython.display import Markdown
from groq import Groq
import requests
import datetime
import json
import os

In [96]:
grouq_api_key="XXXXXXXXXXXXXXXXXX"

weather_api_key = "XXXXXXXXXXXXXXXXXX"
base_url = "http://api.openweathermap.org/"

In [97]:
city_name = "Colombo"
country_code = "+94"
response_limit = 1

url_get_coordinates = f"{base_url}geo/1.0/direct?q={city_name},{country_code}&limit={response_limit}&appid={weather_api_key}"

forecast_response = requests.get(url_get_coordinates)

In [98]:
forecast_data = forecast_response.text
forecast_parsed_data = json.loads(forecast_data)

latitude = forecast_parsed_data[0]['lat']
longitude = forecast_parsed_data[0]['lon']

In [99]:
#Get Weather Forecast
url_weather_data = f"{base_url}data/2.5/weather?lat={latitude}&lon={longitude}&appid={weather_api_key}"
response = requests.get(url_weather_data)

In [100]:
# Convert timestamp to date
date = datetime.datetime.utcfromtimestamp(json.loads(response.text)['dt']).strftime('%Y-%m-%d')

In [101]:
# Get AQI
url_air_quality = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={latitude}&lon={longitude}&appid={weather_api_key}"
air_quality = requests.get(url_air_quality)

air_quality_data = air_quality.text
air_quality_parsed_data = json.loads(air_quality_data)

# Extract air quality info
air_quality_info = {
    "AQI": air_quality_parsed_data['list'][0]['main']['aqi'],
    "Components": air_quality_parsed_data['list'][0]['components'],
    "Timestamp": air_quality_parsed_data['list'][0]['dt']
}

In [102]:
# Get UVI
url_uv_index = f"http://api.openweathermap.org/data/2.5/uvi?lat={latitude}&lon={longitude}&appid={weather_api_key}"
uv_index = requests.get(url_uv_index)

uv_index_data = uv_index.text
uv_index_parsed_data = json.loads(uv_index_data)

uv_index_info = uv_index_parsed_data['value']

In [103]:
model = Groq(api_key=grouq_api_key)

In [104]:
prompt = f"""You are a weather assistant. Your task is to generate a concise weather report based on the provided data. The report must strictly adhere to the following format:
Weather Report for [City Name] Today({date}) \n
  Temperature: [Temperature in Celsius]\n
  Humidity: [Humidity Percentage]\n
  Wind Speed: [Wind Speed in km/h]\n
  Air Quality Index: [AQI Category]\n
  UV Index: [UV Index Value]\n

Recommendations:\n
  [Recommendation 1]\n
  [Recommendation 2]\n
  [Recommendation 3]\n
  More recommendations can be added based on the provided data.

Important Instructions:
  Do not deviate from the format.
  Do not add extra sentences like Here is the weather report based on the provided data:
  Output should be only consist of the report
  Replace placeholders like [City Name], [Temperature in Celsius], etc., with the corresponding data.
  Ensure that recommendations are practical and based on the provided weather conditions.
  Use proper formatting, tabs, alignments and bolding if necessary to highlight important factors

Here is the data:
Weather Forecast: {response.text}
Air Quality Index: {air_quality_info}
UVI: {uv_index_info}
"""


In [105]:
chat_completion = model.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
    stream=False,
)

In [106]:
display(Markdown(chat_completion.choices[0].message.content))

**Weather Report for Colombo Today (2024-12-23)**

  **Temperature:** 31.14°C

  **Humidity:** 65%

  **Wind Speed:** 4.11 km/h

  **Air Quality Index:** **Good** (AQI Category: 3)

  **UV Index:** **High** (9.48)

**Recommendations:**

  * Wear light and breathable clothing to combat the warm weather.
  * Take necessary precautions when stepping out, as the UV index is high.
  * Consider using a sun umbrella or hat to protect yourself from the sun.
  * Keep yourself hydrated by drinking plenty of water throughout the day.

Note: Since the wind speed is relatively low, there's no need to worry about strong gusts or turbulence. However, it's still a good idea to take necessary precautions when stepping out.